In [ ]:
from google.colab import drive
import json
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install transformers

In [ ]:
import pickle
import numpy as np
import argparse
import numpy as np
from collections import Counter
import pdb
import time
import torch
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import datetime
from sklearn.metrics import confusion_matrix, classification_report
from transformers import BartTokenizer, BartForSequenceClassification
import pandas as pd
import random

In [ ]:
#program logic
# preprocess data: train data | validate data
# intialized stage: predict --- > data split
# iteration stage: fine_tune(data_split_output) ---> predict(fine_tune_output) ---> data_split(predict_output)

# **preprocess data stage**

In [ ]:
#preprocess data
from sklearn.model_selection import train_test_split

#data=pd.read_csv('https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv')
data=pd.read_csv('/content/gdrive/MyDrive/fox_news_data.csv')


KeyError: ignored

In [ ]:
data=pd.read_csv('/content/gdrive/MyDrive/fox_news_data.csv')
data['Label']=pd.Series([1 if word==2 else word for word in data['Label']])
data

,Text,Label
0,Merkel would never say NO,0
1,"Expect more and more women to be asking .. ""wh...",0
2,Groping people in public wasn't already illega...,1
3,"Merkel, possible the only person in charge who...",0
4,"They know very well, no means NO! They need to...",0
...,...,...
1523,"No, 10000 Loose nuts off their meds!",0
1524,Just another ugly leftist.,0
1525,"True. Most leftists ,esp female leftists have ...",0
1526,"First, lets get this straight: a white, gay ma...",0


In [ ]:
import nltk
nltk.download('punkt')
data['Text'] = data.apply(lambda row: nltk.word_tokenize(row['Text']), axis=1)
import string
table = str.maketrans('', '', string.punctuation)
# Remove word that is not alphabetic
data['Text'] = [[word for word in element if word.isalpha()] for element in data['Text']]

# remove punctuation from each word
#data['tweet'] = [[word.translate(table) for word in element] for element in data['tweet']]

# Normalize word
data['Text'] = [[word.lower() for word in element] for element in data['Text']]

from nltk.corpus import stopwords
#nltk.download('stopwords')
#stop_words = stopwords.words('english')
#data['tweet'] = [[word for word in element if not word in stop_words] for element in data['tweet']]

from nltk.tokenize.treebank import TreebankWordDetokenizer
data['Text'] = data.apply(lambda row: TreebankWordDetokenizer().detokenize(row['Text']), axis=1)
#sentences=data['tweet']
initial_data = data.copy() #only retrive sentence and label

# train validate split
train_data, valid_data = train_test_split(initial_data,test_size=0.3)
train_data, valid_data = train_data.to_numpy(), valid_data.to_numpy()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#define model
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english", num_labels = 2)
#PATH="/content/gdrive/MyDrive/IDL_project/projectmodel_0408.pt"
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")

#load the model we trained
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.load_state_dict(torch.load(PATH))
model = model.to(device)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device = 0)

# **Initialization Stage**

In [ ]:
#prediction block
#initialized first prediction
sentences,labels = read_data(train_data)
predict_data =predict(sentences, labels)

Finished processing 0/1069 sentences
Finished processing 1000/1069 sentences


In [ ]:
predict_data[:5]

[('to prison', 0, 1, 0.9756612181663513),
 ('first immigrate them then seduce them and then get them into acts where they dont want to hear no and then get them to jail if they are not submissive',
  1,
  0,
  0.8794687986373901),
 ('i agree you know the other thing is this she is the leader of who either woof or agree with all of this garbage does that make her the founder of a hate group and all of her followers members of a hate group last time i checked if a white male posted these things and had such a following certainly this would be considered hate speech would it not',
  0,
  0,
  0.8760384321212769),
 ('yeeeeup actually it a fence is it', 0, 1, 0.969562828540802),
 ('spoken like a true wookie', 0, 0, 0.6462669372558594)]

In [ ]:
#data split block
#initialized stage: data_split
fine_tuned_data,next_iter_data = data_split(predict_data,train_data)

Overall Accuracy: 0.25350795135640786
Check: True
Total Dataset: 1069 Non-hate: 948 Hate: 121
Nonhate Accuracy: 0.23734177215189872
Hate Accuracy: 0.38016528925619836
Number chosen: 47 6
Top10 Confidence Non-hate Accuracy: 0.1702127659574468
Top10 Confidence hate Accuracy: 0.0
Check: True


# **Iteration Stage**

In [ ]:
#ITERATIVELY FINE TUNE MODEL

for i in range(20):
    print("\n========fine tuning stage ============\n")
    fine_tuning(fine_tuned_data, val_file= valid_data,tokenizer = tokenizer, model = model)

    print("\n========predict stage ============\n")
    sentences,labels = read_data(next_iter_data)
    predict_data =predict(sentences, labels)

    print("\n========data split stage ============\n")
    fine_tuned_data,next_iter_data = data_split(predict_data,train_data)


========fine tuning stage ============

Train Label Distribution:
Counter({1: 51})
   51 training samples
Train Loader: 4
Validation Label Distribution:
Counter({1: 324, 0: 135})


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


  459 training samples
Validation Loader: 4

  Average training loss: 0.01
  Training epcoh took: 0:00:02
 Training Accuracy: 1.0000
Time: 1.8189713954925537
 Validation Accuracy: 0.5076
Time: 5.795353651046753

========predict stage ============

Finished processing 0/1035 sentences
Finished processing 1000/1035 sentences

========data split stage ============

Overall Accuracy: 0.46570048309178746
Check: True
Total Dataset: 1035 Non-hate: 345 Hate: 690
Nonhate Accuracy: 0.11014492753623188
Hate Accuracy: 0.6434782608695652
Number chosen: 17 34
Top10 Confidence Non-hate Accuracy: 0.11764705882352941
Top10 Confidence hate Accuracy: 0.23529411764705882
Check: True

========fine tuning stage ============

Train Label Distribution:
Counter({1: 51})
   51 training samples
Train Loader: 4
Validation Label Distribution:
Counter({1: 324, 0: 135})
  459 training samples
Validation Loader: 4

  Average training loss: 0.01
  Training epcoh took: 0:00:01
 Training Accuracy: 1.0000
Time: 0.9199566

# **Helper function**

In [ ]:
def read_data(data):
    
    sentences, labels = [], []

    for item in data:
        #item[0] : sentence
        #item[1] : pred_label
        sentences.append(item[0])
        labels.append(item[1])

    return sentences, labels

In [ ]:
#read_data_test ---check predict.py

In [ ]:
def predict(sentences, labels):
    preds, output_labels = [], []
    pred_label_score = []
    mapping_dict = {'NON_HATE':0,'HATE':1}
    classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device = 0)

    for i, (sentence, label) in enumerate(zip(sentences, labels)):
        try:
            pred = classifier(sentence)
        except:
            print('Faced error in predicting sentiment for: {}. Moving on...'.format(sentence))
            continue

        pred_label = pred[0]['label']

        if pred_label in mapping_dict:
            pred_label = mapping_dict[pred_label]

        # pred_label = pred_label.lower()
        # label = label.lower()
        preds.append(pred_label)
        output_labels.append(label)
        pred_label_score.append((sentence, pred_label, label, pred[0]['score']))

        if i % 1000 == 0:
            print('Finished processing {}/{} sentences'.format(i, len(sentences)))  
              
    return pred_label_score

In [ ]:
def data_split(data, original_train_data):

    n_nonhat, n_hat = count(data)

    overall_accuracy = np.array([pred_label == label for (sentence, pred_label, label, score) in data])
    print('Overall Accuracy:', np.sum(overall_accuracy)/len(data))

    sorted_pred_nonhate, sorted_pred_hate = pred_result_split(data)


    ##Total number of samples in top10% high confidence predictions for each class
    # n_top10_neither = n_nei // 20   #min(args.positive, n_pos) #(n_pos * args.positive)//100
    # n_top10_offensive = n_off // 20   #min(args.negative, n_neg)#(n_neg * args.negative)//100
    n_top10_nonhate = n_nonhat // 20
    n_top10_hate = n_hat // 20    #0#(n_neu * args.neutral)//100

    print('Number chosen:', n_top10_nonhate, n_top10_hate)
    
    
    #Find top 10 percent accuracy
    # neither_top10_accuracy = np.array([pred_label == label for i, (sentence, pred_label, label, score) in enumerate(sorted_pred_neither) if i < n_top10_neither ])
    # offensive_top10_accuracy = np.array([pred_label == label for i, (sentence, pred_label, label, score) in enumerate(sorted_pred_offensive) if i < n_top10_offensive ])
    nonhate_top10_accuracy = np.array([pred_label == label for i, (sentence, pred_label, label, score) in enumerate(sorted_pred_nonhate) if i < n_top10_nonhate ])
    hate_top10_accuracy = np.array([pred_label == label for i, (sentence, pred_label, label, score) in enumerate(sorted_pred_hate) if i < n_top10_hate ])

    print('Top10 Confidence Non-hate Accuracy:', np.sum(nonhate_top10_accuracy) / n_top10_nonhate)
    print('Top10 Confidence hate Accuracy:', np.sum(hate_top10_accuracy) / n_top10_hate)

    
    #Pick top chosen percent
    # top_confidence_neither = sorted_pred_neither[:n_top10_neither]
    # top_confidence_offensive = sorted_pred_offensive[:n_top10_offensive]
    top_confidence_nonhate = sorted_pred_hate[:n_top10_nonhate]
    top_confidence_hate = sorted_pred_hate[:n_top10_hate]

    fine_tune_data = top_confidence_nonhate + top_confidence_hate
    random.shuffle(fine_tune_data)
    print('Check:', len(fine_tune_data) == len(top_confidence_nonhate) + len(top_confidence_hate))

    #Collecting chosen sentences and removing them from original dataset, and saving
    chosen_sentences = [sentence for (sentence, pred_label, label, score) in fine_tune_data]

    next_iteration_data = []
    for (tweet, label) in original_train_data:
        if tweet not in chosen_sentences:
            next_iteration_data.append((tweet, label))

    return fine_tune_data, next_iteration_data

In [ ]:
def count(data):
    predicted_labels = [pred_label for (sentence, pred_label, label, score) in data]
    n_non_hat = 0
    n_hat = 0

    for label in predicted_labels:
        if label == 0:
            n_non_hat += 1
        elif label == 1:
            n_hat += 1

    return n_non_hat, n_hat

In [ ]:
#helper used in data_split function
def pred_result_split(data):
    
    #Store in different dictionaries based on actual label
    # pred_neither = []
    # pred_offensive = []
    pred_nonhate = []
    pred_hate = []

    for (sentence, pred_label, label, score) in data:
        if pred_label == 0:
            pred_nonhate.append((sentence, pred_label, label, score))
        elif pred_label == 1:
            pred_hate.append((sentence, pred_label, label, score))
        
    print('Check:', (len(pred_nonhate) + len(pred_hate)) == len(data))
    print('Total Dataset:', len(data), 'Non-hate:', len(pred_nonhate), 'Hate:', len(pred_hate))
    
    #Original Accuracy for predictions made
    # neither_accuracy = np.array([pred_label == label for i, (sentence, pred_label, label, score) in enumerate(pred_neither)])
    # offensive_accuracy = np.array([pred_label == label for i, (sentence, pred_label, label, score) in enumerate(pred_offensive)])
    nonhate_accuracy = np.array([pred_label == label for i, (sentence, pred_label, label, score) in enumerate(pred_nonhate)])
    hate_accuracy = np.array([pred_label == label for i, (sentence, pred_label, label, score) in enumerate(pred_hate)])

    # print('Neither Accuray:', np.sum(neither_accuracy)/len(pred_neither))
    # print('Offensive Accuracy:', np.sum(offensive_accuracy)/len(pred_offensive))
    print('Nonhate Accuracy:', np.sum(nonhate_accuracy)/len(pred_nonhate))
    print('Hate Accuracy:', np.sum(hate_accuracy)/len(pred_hate))

    ##Sorting by predicting confidence
    sorted_pred_nonhate = sorted(pred_nonhate, key=lambda k: k[3], reverse=True)
    sorted_pred_hate = sorted(pred_hate, key=lambda k: k[3], reverse=True)

    return sorted_pred_nonhate, sorted_pred_hate

**Fine tuning**

In [ ]:
def fine_tuning(fine_tune_file, val_file,tokenizer, model):
    #
    #read tokenizer
    #read model
    sentences, labels = read_data(fine_tune_file) #label is pred_label from the prediction

    label_distribution = Counter(labels)
    print('Train Label Distribution:')
    print(label_distribution)

    batch_size = 16
    num_epochs = 1
    train_dataloader = create_loader(tokenizer, sentences, labels, batch_size) #create train loader
    print('Train Loader:', len(train_dataloader))

    optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
    total_steps = len(train_dataloader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
    
    #read validate data
    sentences, labels = read_data(val_file)
    label_distribution = Counter(labels)
    print('Validation Label Distribution:')
    print(label_distribution)

    batch_size = 128
    val_dataloader = create_loader(tokenizer, sentences, labels, batch_size)
    print('Validation Loader:', len(val_dataloader))

    #fine tuning loop
    for i in range(num_epochs):
        train_epoch(train_dataloader, model, optimizer, scheduler)
        eval_epoch(val_dataloader, model)

    #save model
    #save tokenizer

In [ ]:
def tokenize_text(tokeinzer, sentences, labels):

    data_input_ids, data_attention_masks, output_labels = [], [], []

    #TODO: can this be optimized?
    max_num_tokens = 0
    for sent, label in zip(sentences, labels):
        tokenized_output = tokeinzer(sent)
        sent_input_ids = torch.tensor(tokenized_output['input_ids'])
        sent_attention_mask = torch.tensor(tokenized_output['attention_mask'])
        if len(sent_input_ids) > max_num_tokens:
            max_num_tokens = len(sent_input_ids)

    for sent, label in zip(sentences, labels):
        tokenized_output = tokeinzer(sent, max_length=max_num_tokens, pad_to_max_length = True)
        sent_input_ids = torch.tensor(tokenized_output['input_ids'])
        sent_attention_mask = torch.tensor(tokenized_output['attention_mask'])
        data_input_ids.append(sent_input_ids)
        data_attention_masks.append(sent_attention_mask)
        output_labels.append(label)

    # Convert the lists into tensors.
    data_input_ids = torch.stack(data_input_ids, dim=0)
    data_attention_masks = torch.stack(data_attention_masks, dim=0)
    output_labels = torch.tensor(output_labels)

    return data_input_ids, data_attention_masks, output_labels

In [ ]:
def create_loader(tokenizer, sentences, labels, batch_size):

    input_ids, attention_masks, labels = tokenize_text(tokenizer, sentences, labels)

    dataset = TensorDataset(input_ids, attention_masks, labels)

    train_size = len(dataset)

    train_dataset = dataset

    print('{:>5,} training samples'.format(train_size))

    train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size 
        )

    return train_dataloader

In [ ]:
def train_epoch(train_dataloader, model, optimizer, scheduler):

    t0 = time.time()
    model.train()
    total_train_loss = 0

    correct = 0
    total = 0

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_size = b_labels.size(0)

        model.zero_grad()
        # pdb.set_trace()
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        
        loss = outputs.loss
        logits = outputs.logits
        total_train_loss += loss.item()
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        smax = torch.softmax(logits, dim = 1)
        indices = torch.argmax(smax, dim = 1)
        correct += torch.sum(indices==b_labels).item()
        total += b_size


    avg_train_loss = total_train_loss / len(train_dataloader)  
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    val_accuracy = correct / total
    print(" Training Accuracy: {0:.4f}".format(val_accuracy))
    print('Time:', time.time() - t0)

In [ ]:
def eval_epoch(validation_dataloader, model):
    t0 = time.time()
    model.eval()

    correct = 0
    total = 0

    for i, batch in enumerate(validation_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_size = b_labels.size(0)

        with torch.no_grad():
            outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        loss = outputs.loss
        logits = outputs.logits
        smax = torch.softmax(logits, dim = 1)
        indices = torch.argmax(smax, dim = 1)

        correct += torch.sum(indices==b_labels).item()
        total += b_size

    val_accuracy = correct / total
    print(" Validation Accuracy: {0:.4f}".format(val_accuracy))
    print('Time:', time.time() - t0)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))